In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

##### Team Members-
##### Name: Ekata S 
##### Name: Lokesh G
##### Name: Kushal S
##### Name: Nagaveni R M 

In [ ]:
import matplotlib.pyplot as plt
%pylab inline
import matplotlib.patches as patches
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import seaborn as sns
import datetime
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [ ]:
#Reading data files:
#Reading players.csv file
players=pd.read_csv("../input/nfl-big-data-bowl-2021/players.csv")
#Reading plays.csv file
plays=pd.read_csv("../input/nfl-big-data-bowl-2021/plays.csv")
#Reading games.csv file
games = pd.read_csv("../input/nfl-big-data-bowl-2021/games.csv")
#Reading week1.csv files
week1=pd.read_csv("../input/nfl-big-data-bowl-2021/week1.csv",index_col=None,header=0)
week2 = pd.read_csv("../input/nfl-big-data-bowl-2021/week2.csv",index_col=None,header=0)
dis = pd.read_csv("../input/weeks-data/week_data.csv",index_col=None,header=0)
weeks = [week1,week2]
week_data_df = pd.concat(weeks,ignore_index=True,axis=0)

In [ ]:
#In height columns there are some cells with "-" which is not according to the format. 
players['height'] = [int(x[0])*12 + int(x[2]) if '-' in x else x for x in players['height']]
players['height'] = pd.to_numeric(players['height'])
#Changing the birthDate format
players['birthDate'] = pd.to_datetime(players['birthDate']).dt.strftime('%m/%d/%Y')
players['birthDate'] = pd.to_datetime(players['birthDate'])

In [ ]:
df = pd.DataFrame(players)


In [ ]:
#filtering so I can see only secondary players
DB_pos = ['CB','SS','FS','DB']
df.loc[df['position'].isin(DB_pos)]
df = df.loc[df['position'].isin(DB_pos)]

In [ ]:
#Removing the rows with missing values
data_without_missing_values = week1.dropna(axis=1)
week1.isnull().sum()

In [ ]:
week1 = week1.drop(['dir','nflId','o','jerseyNumber','route','time','x','y','event','frameId','team','gameId','playId','playDirection'],axis=1)
week1

# Data Visualization and EDA

In [ ]:
#Based on the position of players to know which player is going to play as Offense, Defense or Special team
cat_item = {'Offense' : ['QB','RB','FB','WR','TE','HB'],
            'Defense' : ['OLB','MLB','LB','ILB','CB','DE','DT','NT','DB','S','SS','FS'],
            'Special' : ['K','P','LS']}
items = {w : k for k,v in cat_item.items() for w in v}
players['teamRole'] = players['position'].map(items)

In [ ]:
# Lets See how many players each college has (based on teamRole)
college = players.groupby(['collegeName','teamRole'], as_index=False)[['nflId']].count()
college.head()
# As shown below the highest number of players are in Alabama college playing in Defense Team Role are 21

# We can see the height of players is between 72 inches to 74 inches

In [ ]:
# Height Ranges
height = players.groupby(['height'], as_index=False)[['nflId']].count()
plt.bar(height['height'],height['nflId'])
plt.savefig('fig-1.jpg')

In [ ]:
#Figuring out the team that obtain most yards in the whole season
yards_total = plays.groupby('possessionTeam').agg({'offensePlayResult': 'sum', 'gameId':'nunique'}).reset_index().rename(columns={'gameId': 'number of matches', 'offensePlayResult': 'yardsGained'})
yards_total['averageYards']= np.round(yards_total['yardsGained']/yards_total['number of matches'])

In [ ]:
I = []
for date in players['birthDate']:
    I.append(int(str(date).split("-")[0])) 
age = []
for k in I:
    age.append((2020 - k))
players['age'] = age
players = players.drop(columns='birthDate')
players = players.drop(columns='nflId')

In [ ]:
#To see which dropback type would result in most yards by considering some factors
dropbacks_total = plays.groupby('typeDropback')['offensePlayResult'].agg(yardsGained='sum', popularity='count')
dropback_passRes = plays.groupby(['typeDropback', 'passResult'])[['passResult']].count()
dropback_passRes = dropback_passRes.add_suffix('_Count').reset_index()
success_dropback = dropback_passRes.loc[dropback_passRes['passResult']== 'C']

mapping = dict(success_dropback[['typeDropback', 'passResult_Count']].values)
dropbacks_total['succesful'] =dropbacks_total.index.map(mapping)
dropbacks_total['averageYard'] = np.round(dropbacks_total['yardsGained']/dropbacks_total['popularity'])
dropbacks_total['success_normal'] = dropbacks_total['succesful']/dropbacks_total['popularity']
dropbacks_total = dropbacks_total.reset_index()
dropbacks_total

In [ ]:
#Checking the correlation between variables
%matplotlib inline
corr = plays.corr()
# sns.heatmap(corr, 
#         xticklabels=corr.columns,
#         yticklabels=corr.columns)

# If defense play result improves,play result also improves


In [ ]:
%matplotlib inline
corr = players.corr()
# sns.heatmap(corr, 
#         xticklabels=corr.columns,
#         yticklabels=corr.columns)


#Height and weight are positively correlated 
# plt.savefig('fig-4.jpg')

In [ ]:
# Relation Between weight and height
# sns.relplot(x='weight',y='height',data=players)
# plt.savefig('fig-5.jpg')

In [ ]:
# Relation between speed and acceleration
# sns.relplot(x='s',y='a',hue='dis',data=week1)
# plt.savefig('fig-6.jpg')

In [ ]:
df = pd.DataFrame(players)
df1 = pd.DataFrame(week1)

In [ ]:
#Filtering to capture only defensive backs(DB)
DB_Pos = ['CB','SS','FS','DB']
df1.loc[df1['position'].isin(DB_Pos)]
df1 = df1.loc[df1['position'].isin(DB_Pos)]

#average speed and acceleration for all DB's
df1.groupby('displayName').mean('s')
df1 = df1.groupby('displayName').mean('s')

In [ ]:
#scatter of speed and acceleration
# ax1 = df1.plot.scatter(x='s', y='a', c='Green')
#3 variable scatter plot
# sns.scatterplot(x= df1['s'], y = df1['a'], hue = df1['dis'])

In [ ]:
#Renaming columns so that it can be merged
df.rename(columns={'displayName' : 'displayName'})
df= df.rename(columns={'displayName' : 'displayName'})
#Resetting index 
df1.reset_index(inplace = True)

In [ ]:
display(df,df1)

In [ ]:
#merging both data sets together
df2 = pd.merge(df,df1)

# Learning model

### Is there anything about a player – for example, their height, weight, experience, speed, or position – that can be used to predict their performance on defense?

In [ ]:


# plt.scatter(df2['weight'],df2['s'])
# plt.xlabel('Weight')
# plt.ylabel('Speed')
# plt.savefig('fig-7.jpg')
# plt.scatter(df2['height'],df2['s'])
# plt.xlabel('Height')
# plt.ylabel('Speed')
# plt.savefig('fig-8.jpg')
# plt.scatter(df2['height'],df2['s'])
# plt.xlabel('Height')
# plt.ylabel('Speed')
# plt.savefig('fig-8.jpg')
# plt.scatter(df2['age'],df2['s'])
# plt.xlabel('Age')
# plt.ylabel('Speed')
# plt.savefig('fig-9.jpg')

In [ ]:
X = df2[['height','weight']]
Y = df2['s']

In [ ]:
#Random Forest Classifier Model using age as dependent
##Train 80% Test 20%
x2 = df2[['height','weight','a','s']]
y2 = df2['age']

In [ ]:
#Test 80% Train20%
from sklearn.model_selection import train_test_split
x2_train, x2_test, y2_train,y2_test = train_test_split(x2,y2,test_size = 0.2)

In [ ]:
#Importing RandomForestClassifier using 100 trees
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(100)
model.fit(x2_train, y2_train)

In [ ]:
#Importing RandomForestClassifier using 100 trees
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(100)
model.fit(x2_train, y2_train)

In [ ]:
y2_predicted = model.predict(x2_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y2_test,y2_predicted)
cm

In [ ]:
# import matplotlib.pyplot as plt
# import seaborn as sn
# plt.figure(figsize=(10,8))
# sn.heatmap(cm, annot=True)
# plt.xlabel('prediction')
# plt.ylabel('result')
# plt.savefig('fig-10.jpg')

In [ ]:
#RandomForestClassifierModel using acceleration as dependent
##Train 80% Test 20%
x3 = df2[['height','weight','a','s','dis']]
y3 = df2['age']

#Test 80% Train20%
from sklearn.model_selection import train_test_split
x3_train, x3_test, y3_train,y3_test = train_test_split(x3,y3,test_size = 0.2)

#Importing RandomForestClassifier using 100 trees
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(100)
model.fit(x3_train, y3_train)

In [ ]:
model.score(x3_test,y3_test)

In [ ]:
y3_predicted = model.predict(x3_test)
from sklearn.metrics import confusion_matrix
cm1 = confusion_matrix(y3_test,y3_predicted)
cm1

In [ ]:
#Much better performance when accounting for distance traveled
# import matplotlib.pyplot as plt
# import seaborn as sn
# plt.figure(figsize=(10,7))
# sn.heatmap(cm1, annot=True)
# plt.xlabel('prediction')
# plt.ylabel('result')

In [ ]:
#Random Forest Classifier Model of top 20 NFL players to try and predict age

x4 = df2[['height','weight','a','dis']]
y4 = df2['s']
y4

In [ ]:
#Test 30% Train 70%
#Increased test size due to size of data being so small
from sklearn.model_selection import train_test_split
x4_train, x4_test, y4_train,y4_test = train_test_split(x4,y4,test_size = 0.3)

In [ ]:
#Importing RandomForestRegressor using 100 trees
from sklearn.ensemble import RandomForestRegressor
model1 = RandomForestRegressor(100)
model1.fit(x4_train, y4_train)

In [ ]:
#This model performed exceptionally well in predicting 
model1.score(x4_test,y4_test)

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn import tree, metrics, model_selection, preprocessing

# select features
y = df2['s'] #Dependent feature
X = df2[['height', 'weight','a','dis']] #Independent features (subset)

# split data randomly into 70% training and 30% test
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    test_size=0.3, random_state=0)

# train the decision tree
dtree = tree.DecisionTreeRegressor( max_depth=5)
dtree.fit(X_train, y_train)



In [ ]:
y_pred = dtree.predict(X_test)

#split dataset in features and target variable
feature_cols = ['height', 'weight', 'a','dis']
X = df2[feature_cols] # Features
y = df2.s # Target variable
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2) # 70% training and 30% test

In [ ]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import metrics
from sklearn import tree,metrics,model_selection,preprocessing

In [ ]:
y = df2['s']
x = df2[['height','weight','a','dis']]
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(x,y,test_size=0.3,random_state=0)
dtree = tree.DecisionTreeRegressor(max_depth=5)
dtree.fit(X_train,Y_train)

In [ ]:
y_pred = dtree.predict(X_test)
y_pred = y_pred.astype(int)
test_arr = []
for i in Y_test:
    test_arr.append(i)
test_arr = np.array(test_arr)
test_arr = test_arr.astype(int)
accuracy = metrics.accuracy_score(test_arr, y_pred)
accuracy = accuracy * 100
print('Accuracy: {:.2f}'.format(accuracy))

XGBoost

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
Independent_features = df2[['height', 'weight','a','dis']]
Dependent_features = df2['s']

E_train,E_test,F_train,F_test = train_test_split(Independent_features,Dependent_features,test_size=0.2,random_state=1)

E_train = E_train.astype(int)
F_train = F_train.astype(int)
F_test = F_test.astype(int)

# Running Various Models
models = []
models.append(('LogisticRegression', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('SVM', SVC()))
models.append(('XGB',XGBClassifier(eta=0.01,gamma=10))) #eta = 0.01,gamma = 10
import time
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name,model in models:
    start_time = time.time()
    model.fit(E_train,F_train)

    F_pred = model.predict(E_test)
    predictions = [round(value) for value in F_pred]


    # evaluate predictions
    accuracy = metrics.accuracy_score(F_test,predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0),name)
    print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import numpy as np 
import pandas as pd
import re
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.patches as mpatches
from plotly.subplots import make_subplots
import random 
import csv
pd.options.mode.chained_assignment = None 

### Which players are the best at closely tracking receivers as they try to get open?

Prescriptive Analysis:

The line graph (Fig-2) below illustrates the performance of players at closely tracking receivers as they try to get open by scaling distance between ball and defensive players.

    Darius Leonard was the first best player at closely tracking receivers as they try to get open in NFL, 2018 Season. Darius Leonard played as Defense Line backer at age of 25 years, with height and weight of 74 inches and 230 pounds. He was form South Carolina State college with Avgerage speed and acceleration of 2.934 yards/second and 1.929 yard/second^2. Darius Leonard covered total distance of 10147.00 yards in NFL,2018 Season.

    Demario Davis was the second best player at closely tracking receivers as they try to get open in NFL, 2018 Season. Demario Davis played as Defense Inline backer at age of 31 years, with height and weight of 74 inches and 248 pounds. He was form Arkansas State college with Avgerage speed and acceleration of 2.757 yards/second and 1.893 yard/second^2. Demario Davis covered total distance of 9704.07 yards in NFL,2018 Season.

    Luke Kuechly was the third best player at closely tracking receivers as they try to get open in NFL, 2018 Season. Luke Kuechly played as Defense Middleline backer at age of 29 years, with height and weight of 75 inches and 238 pounds. He was form Boston College with Avgerage speed and acceleration of 2.832 yards/second and 1.868 yard/second^2. Luke Kuechly covered total distance of 9923.56 yards in NFL,2018 Season.


In [ ]:
players=pd.read_csv("../input/nfl-big-data-bowl-2021/players.csv")

In [ ]:
player_game_stats = pd.DataFrame()
week_data_df = week_data_df[week_data_df['displayName']!='Football']
week_data_df['jerseyNumber'] = week_data_df['jerseyNumber'].astype(np.int8)
week_data_df['nflId'] = week_data_df['nflId'].astype(np.int64)
player_game_stats = player_game_stats.append(week_data_df.groupby(['gameId','nflId'],as_index=False).agg({'s':'mean','a':'mean','dis':'sum'}))
stat = player_game_stats.groupby(by=['nflId'])['s','a'].mean()
displacement = player_game_stats.groupby(by=['nflId'])['dis'].sum()
players['avgSpeed'] = 0.0
players['avgAcceleration'] = 0.0
players['totalDisplacement'] = 0.0
for i in range(len(players)):
    try:
        if stat.s.loc[players.nflId[i]]:
            players['avgSpeed'][i] = stat.s.loc[players.nflId[i]]
        if stat.a.loc[players.nflId[i]]:
            players['avgAcceleration'][i] = stat.a.loc[players.nflId[i]]
        if displacement.loc[players.nflId[i]]:
            players['totalDisplacement'][i] = displacement.loc[players.nflId[i]]
    except :
        continue
temp3 = dis.groupby(by=['gameId','playId','frameId']).distanceBetweenBall.min()
temp3 = pd.DataFrame(temp3)
temp3.reset_index(level=['gameId','playId','frameId'],inplace= True)
temp3 = temp3.merge(dis, left_on=['gameId','playId','frameId','distanceBetweenBall' ], right_on=['gameId','playId','frameId','distanceBetweenBall'], how='left' )
temp3.drop(columns=['time', 'x', 'y',
        'a', 'dis', 'o', 'dir', 'event','team', 'playDirection', 'route', 'unit','s',
       'week', 'position','displayName'], axis=1, inplace=True)
temp3['nflId'] = temp3['nflId'].astype('int64') 
temp3 = temp3.merge(players, left_on=['nflId'], right_on=['nflId'], how='left' )
temp3.drop(columns=["birthDate","Unnamed: 0"], axis=1, inplace = True)
df = temp3.displayName.value_counts().nlargest(15)
df = pd.DataFrame(df)
df['displayName'] = (df['displayName']/ len(temp3) )*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.displayName,marker_symbol = "star",
                    mode='lines+markers',
                    name='lines+markers',line=dict(color='blue', width=5), marker_size=20))
fig.update_layout(
    title="Best defensive player at closely tracking receivers as they try to get open in NFL, 2018 Season",
    xaxis_title="Player Names  \n",
    yaxis_title="Metric score" )

fig.show()
plt.savefig('fig-2.jpg')

### Which players are the best at defending pass plays when the ball arrives

Prescriptive Analysis:

The line graph (Fig-3) below illustrates the preformance of players at defending pass plays when the ball arrives by scaling the distance between ball and defensive player at the event of pass arrives / ball arrives in NFL, 2018 season.

    Darius Leonard was the first best player at defending pass plays when ball arrives in NFL, 2018 Season. Darius Leonard played as Defense Line backer at age of 25 years, with height and weight of 74 inches and 230 pounds. He was form South Carolina State college with Avgerage speed and acceleration of 2.934 yards/second and 1.929 yard/second^2. Darius Leonard covered total distance of 10147.00 yards in NFL,2018 Season.

    Leighton Vander Esch was the second best player at defending pass plays when ball arrives in NFL, 2018 Season. Leighton Vander Esch played as Defense Line backer at age of 24 years, with height and weight of 74 inches and 230 pounds. He was form Boise State college with Avgerage speed and acceleration of 2.88 yards/second and 1.92 yard/second^2. Leighton Vander Esch covered total distance of 8566.81 yards in NFL,2018 Season.

    Kendall Fuller was the third best player at defending pass plays when ball arrives in NFL, 2018 Season. Kendall Fuller played as Defense Corner backer at age of 25 years, with height and weight of 71 inches and 198 pounds. He was form Virginia Tech college with Avgerage speed and acceleration of 3.19 yards/second and 1.93 yard/second^2. Kendall Fuller covered total distance of 12957.60 yards in NFL,2018 Season.


In [ ]:
temp1  = dis[dis.event == "pass_arrived"]
temp1 = temp1.groupby(by=['gameId','playId', 'frameId']).distanceBetweenBall.min()
temp1 = pd.DataFrame(temp1)
temp1.reset_index(level=['gameId','playId','frameId'],inplace= True)
temp1 = temp1.merge(dis, left_on=['gameId','playId','frameId','distanceBetweenBall' ], right_on=['gameId','playId','frameId','distanceBetweenBall'], how='left' )
temp1.drop(columns=['time', 'x', 'y',
        'a', 'dis', 'o', 'dir','team', 'playDirection', 'route', 'unit','s',
       'week', 'position','displayName'], axis=1, inplace=True)
temp1['nflId'] = temp1['nflId'].astype('int64') 
temp1 = temp1.merge(players, left_on=['nflId'], right_on=['nflId'], how='left' )
temp1.drop(columns=["birthDate","Unnamed: 0"], axis=1, inplace = True)

df = temp1.displayName.value_counts().nlargest(15)
df = pd.DataFrame(df)
df['displayName'] = (df['displayName']/ len(temp1) )*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.displayName,marker_symbol = "star",
                    mode='lines+markers',
                    name='lines+markers',line=dict(color='black', width=5), marker_size=20))
fig.update_layout(
    title="Best defensive player at defending pass plays when the ball arrives in NFL, 2018 Season",
    xaxis_title="Player Names  \n",
    yaxis_title="Metric score" )

fig.show()
plt.savefig('fig-3.jpg')

### Who are the NFL's best players against the pass

Prescriptive Analysis:

The line graph (Fig-4) below describes the preformance of players against the passes in NFL, 2018 season by scaling the distance between ball and defense players at the event of pass outcome interception, pass outcome incomplete.

    Steven Nelson was the first best player against the pass in NFL, 2018 Season. Steven Nelson was form Oregon State college at age of 27. He played as Defense Corner backer with height anf weight of 71 inches and 194 pounds. Steven Nelson had totally covered distance of 13950.07 yards with average speed and acceleration of 3.24 yards/second and 1.87 yards/second^2.
    Jalen Ramsey was the second best player against the pass in NFL, 2018 Season. Jalen Ramsey was form Florida State college at age of 26. He played as Defense Corner backer with height and weight of 73 inches and 203 pounds. Jalen Ramsey had totally covered distance of 11455.19 yards with average speed and acceleration of 3.124 yards/second and 1.88 yards/second^2.
    Jalen Ramsey was the third best player against the pass in NFL, 2018 Season. Jalen Ramsey was form Samford college at age of 27. He played as Defense Corner backer with height anf weight of 73 inches and 212 pounds. Jalen Ramsey had totally covered distance of 10954.13 yards with average speed and acceleration of 2.99 yards/second and 1.86 yards/second^2.

Overall all best 20 players against the pass results be playing Defense Corner Backer position in NFL, 2018 season.


In [ ]:
temp2 = dis[(dis.event == "pass_outcome_incomplete") | (dis.event == "pass_outcome_interception") ]
temp2 = temp2.groupby(by=['gameId','playId', 'frameId']).distanceBetweenBall.min()
temp2 = pd.DataFrame(temp2)
temp2.reset_index(level=['gameId','playId','frameId'],inplace= True)
temp2 = temp2.merge(dis, left_on=['gameId','playId','frameId','distanceBetweenBall' ], right_on=['gameId','playId','frameId','distanceBetweenBall'], how='left' )
temp2.drop(columns=['time', 'x', 'y',
        'a', 'dis', 'o', 'dir','team', 'playDirection', 'route', 'unit','s',
       'week', 'position','displayName'], axis=1, inplace=True)
temp2['nflId'] = temp2['nflId'].astype('int64') 
temp2 = temp2.merge(players, left_on=['nflId'], right_on=['nflId'], how='left' )
temp2.drop(columns=["birthDate","Unnamed: 0"], axis=1, inplace = True)

df = temp2.displayName.value_counts().nlargest(15)
df = pd.DataFrame(df)
df['displayName'] = (df['displayName']/ len(temp2) )*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.displayName,marker_symbol = "star",
                    mode='lines+markers',
                    name='lines+markers',line=dict(color='orange', width=5), marker_size=20))
fig.update_layout(
    title="Best defensive players against the pass in NFL, 2018 Season",
    xaxis_title="Player Names  \n",
    yaxis_title="Metric score" )

fig.show()
plt.savefig('fig-4.jpg')

### Best defensive players in tackling offensive players

The line graph below describes the performance of players in tackling in NFL, 2018
season by scaling the distance between ball and defense players at the event of tackle.
* Darius Leonard was the first best player against the pass in NFL
* Leighton Vander Esch was the second best player
* Roquan Smith was the third best player against the pass in NFL

In [ ]:
temp2 = dis[(dis.event == "tackle") ]
temp2 = temp2.groupby(by=['gameId','playId', 'frameId']).distanceBetweenBall.min()
temp2 = pd.DataFrame(temp2)
temp2.reset_index(level=['gameId','playId','frameId'],inplace= True)
temp2 = temp2.merge(dis, left_on=['gameId','playId','frameId','distanceBetweenBall' ], right_on=['gameId','playId','frameId','distanceBetweenBall'], how='left' )
temp2.drop(columns=['time', 'x', 'y',
        'a', 'dis', 'o', 'dir','team', 'playDirection', 'route', 'unit','s',
       'week', 'position','displayName'], axis=1, inplace=True)
temp2['nflId'] = temp2['nflId'].astype('int64') 
temp2 = temp2.merge(players, left_on=['nflId'], right_on=['nflId'], how='left' )
temp2.drop(columns=["birthDate","Unnamed: 0"], axis=1, inplace = True)

df = temp2.displayName.value_counts().nlargest(15)
df = pd.DataFrame(df)
df['displayName'] = (df['displayName']/ len(temp2) )*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df.displayName,marker_symbol = "star",
                    mode='lines+markers',
                    name='lines+markers',line=dict(color='darkblue', width=5), marker_size=20))
fig.update_layout(
    title="Best defensive players in tackling offensive players in NFL, 2018 Season",
    xaxis_title="Player Names  \n",
    yaxis_title="Metric score" )

fig.show()
plt.savefig('fig-5.jpg')